# CityIQ Data Pipeline and Spatial Aggregation

This notebook loads and trasforms CityIQ metadata and asset events data and maps a summary of the pedestrian events over the last hour

### Import Python modules

In [3]:
# import cartoframes modules
from cartoframes import read_carto, to_carto
from cartoframes.auth import set_default_credentials
from cartoframes.viz import *
from cartoframes.data.observatory import Catalog, Geography

# shapely
from shapely import wkt

# import pandas and geopandas modules
import pandas as pd
import geopandas as gpd

# import CityIq and modules for timing
from cityiq import CityIq
import time
import json
from dateutil import rrule
from datetime import datetime
from dateutil.relativedelta import *

### Set up CARTO credentials 

In [4]:
set_default_credentials('creds_usignite.json')

### Set start and end times to pull data for the last month

In [5]:
# set time frame for use when querying for events (epoch time in milliseconds)
now = datetime.now()
past = now - relativedelta(months=1)

### Set CityIQ token and pull sensor metadata for pedestrian events types

In [6]:
# get CityIq token
myCIQ = CityIq("City")
myCIQ.fetchToken()

# getting assets - assets with PEDEVT events
myCIQ.fetchMetadata("assets","pedestrian","eventTypes:PEDEVT")
san_diego_pedestrian_sensor_metadata = myCIQ.getAssets()

More Metadata assets are available for this query.
TotalElements: 232. Your specified size: 100


*   Make a Dataframe from returned sensor metadata
*   Drop null records 
*   Split string for coordinates into latitude and longitude and store in a property
*   Make a GeoDataFrame with latitude and longitude data

In [7]:
san_diego_pedestrian_sensor_metadata_df = pd.DataFrame(san_diego_pedestrian_sensor_metadata)
san_diego_pedestrian_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_pedestrian_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 

san_diego_pedestrian_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_pedestrian_sensor_metadata_df["longitude"]= latlng[1].astype(float)

san_diego_pedestrian_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_pedestrian_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_pedestrian_sensor_metadata_df.longitude, san_diego_pedestrian_sensor_metadata_df.latitude))
san_diego_pedestrian_sensor_metadata_gdf.head()

,assetUid,parentAssetUid,eventTypes,mediaType,assetType,coordinates,status,latitude,longitude,geometry
0,000223ee-a868-474b-abcb-12ff1bad00a3,131d67b7-daaf-4252-9202-5df347a6c2a3,"[PKOUT, PKIN, BICYCLE, PEDEVT]","IMAGE,VIDEO",CAMERA,32.71143062:-117.1600173,ONLINE,32.711431,-117.160017,POINT (-117.16002 32.71143)
1,02ded76c-a44d-45ab-bb6d-df58d2be33eb,b2dddbf2-1036-4f50-b743-145415113ddc,"[TFEVT, PEDEVT, PKOUT, PKIN]","IMAGE,VIDEO,VIDEO_STREAM",CAMERA,32.72292597:-117.1568986,OFFLINE,32.722926,-117.156899,POINT (-117.15690 32.72293)
2,03affc64-5fdc-426c-98bb-8571da568449,f4373d1d-4055-4b31-9dab-0dcabf0e85fd,"[PKOUT, PKIN, BICYCLE, PEDEVT]","IMAGE,VIDEO,VIDEO_STREAM",CAMERA,32.71163629:-117.154756,ONLINE,32.711636,-117.154756,POINT (-117.15476 32.71164)
3,052a12aa-1672-4593-98e3-1a14c0435a4d,63904ebb-0e00-4f16-9e52-d7dee9e18e5d,"[BICYCLE, PEDEVT]","IMAGE,VIDEO",CAMERA,32.76087897:-117.1501676,ONLINE,32.760879,-117.150168,POINT (-117.15017 32.76088)
4,05330a7b-5b72-4135-9680-b650ede1cb32,4b45e488-64d6-4e40-82e4-62536b649be6,"[PKOUT, PKIN, PEDEVT, BICYCLE]","IMAGE,VIDEO,VIDEO_STREAM",CAMERA,32.70948112:-117.1559482,ONLINE,32.709481,-117.155948,POINT (-117.15595 32.70948)


In [8]:
Layer(san_diego_pedestrian_sensor_metadata_gdf)

## Pull census tract geographies from Data Observatory and spatially aggregate assets to tracts

In [9]:
geographies = Catalog().country('usa').category('demographics').geographies
geographies

[<Geography.get('mbi_blockgroups_1ab060a')>,
 <Geography.get('mbi_counties_141b61cd')>,
 <Geography.get('mbi_county_subd_e8e6ea23')>,
 <Geography.get('mbi_pc_5_digit_4b1682a6')>,
 <Geography.get('usct_blockgroup_f45b6b49')>,
 <Geography.get('usct_cbsa_6c8b51ef')>,
 <Geography.get('usct_censustract_bc698c5a')>,
 <Geography.get('usct_congression_b6336b2c')>,
 <Geography.get('usct_county_ec40c962')>,
 <Geography.get('usct_county_92f1b5df')>,
 <Geography.get('usct_place_12d6699f')>,
 <Geography.get('usct_puma_b859f0fa')>,
 <Geography.get('usct_schooldistr_515af763')>,
 <Geography.get('usct_schooldistr_da72a4cb')>,
 <Geography.get('usct_schooldistr_287be4f7')>,
 <Geography.get('usct_state_4c8090b5')>,
 <Geography.get('usct_zcta5_75071016')>]

In [10]:
geography = Geography.get('usct_censustract_bc698c5a')
geography_df = geography.to_dataframe()
geography_df.head()

,geoid,do_label,do_area,do_perimeter,do_num_vertices,geom
0,6037123206,1232.06,602907.522,3130.340,5,"POLYGON((-118.379046 34.190309, -118.370314 34..."
1,6037127804,1278.04,638163.037,3195.499,5,"POLYGON((-118.466187 34.193917, -118.457453 34..."
2,42101002801,28.01,222964.493,2190.300,5,"POLYGON((-75.159204 39.925511, -75.156095 39.9..."
3,36047102000,1020.00,156762.634,1613.701,5,"POLYGON((-73.890328 40.63908, -73.886646 40.63..."
4,17031020801,208.01,655600.964,3239.043,5,"POLYGON((-87.699157 41.983121, -87.689448 41.9..."


Filter for Census tracts in San Diego Metro Area. 

In [11]:
geography_df['geoid'] = geography_df['geoid'].apply(str)
sd_tracts = geography_df[geography_df['geoid'].str.startswith("6073")]
sd_tracts.head()

,geoid,do_label,do_area,do_perimeter,do_num_vertices,geom
91,6073000800,8.00,633133.730,3210.687,9,"POLYGON((-117.146314 32.746708, -117.146194 32..."
167,6073008003,80.03,752320.880,3584.317,12,"POLYGON((-117.259975 32.806597, -117.260911 32..."
237,6073001500,15.00,793607.042,3759.341,14,"POLYGON((-117.128223 32.741031, -117.125006 32..."
259,6073011100,111.00,908767.319,4064.244,15,"POLYGON((-117.185431 32.695361, -117.176495 32..."
351,6073008506,85.06,1288668.282,4914.219,19,"POLYGON((-117.194813 32.831565, -117.194675 32..."


In [12]:
sd_tracts['geom'] = sd_tracts['geom'].apply(wkt.loads)
sd_tracts_gdf = gpd.GeoDataFrame(sd_tracts, geometry='geom')
Layer(sd_tracts_gdf)

/Users/jd/Documents/USIgnite/Portland_CARTO_notebook/env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Use sjoin to spatial join census tract geoids to assetids

In [13]:
assets_within_tracts = gpd.sjoin(san_diego_pedestrian_sensor_metadata_gdf, sd_tracts_gdf, how="inner", op='within')
assets_within_tracts.head()

,assetUid,parentAssetUid,eventTypes,mediaType,assetType,coordinates,status,latitude,longitude,geometry,index_right,geoid,do_label,do_area,do_perimeter,do_num_vertices
0,000223ee-a868-474b-abcb-12ff1bad00a3,131d67b7-daaf-4252-9202-5df347a6c2a3,"[PKOUT, PKIN, BICYCLE, PEDEVT]","IMAGE,VIDEO",CAMERA,32.71143062:-117.1600173,ONLINE,32.711431,-117.160017,POINT (-117.16002 32.71143),18579,6073005400,54.0,1541644.704,9342.847,61
5,0ad510cd-29ea-4ea6-8bf4-aae29cb46a8d,fdae3913-a898-4005-8f6e-d0fea235020f,"[PEDEVT, BICYCLE]","IMAGE,VIDEO,VIDEO_STREAM",CAMERA,32.71800083:-117.1683939,ONLINE,32.718001,-117.168394,POINT (-117.16839 32.71800),18579,6073005400,54.0,1541644.704,9342.847,61
6,0b07deea-6d66-4cbd-9824-c29e93730305,6e094ebd-a7ee-4926-92ce-ce0ae5a5134a,"[PKOUT, PKIN, PEDEVT, BICYCLE]","IMAGE,VIDEO,VIDEO_STREAM",CAMERA,32.71035993:-117.1591058,ONLINE,32.710360,-117.159106,POINT (-117.15911 32.71036),18579,6073005400,54.0,1541644.704,9342.847,61
18,12dee1db-5de1-4403-90af-d3cfb1f8d991,415d861c-eec0-4db6-9a48-ff4b7c096239,"[BICYCLE, PEDEVT]","IMAGE,VIDEO,VIDEO_STREAM",CAMERA,32.71606122:-117.168402,ONLINE,32.716061,-117.168402,POINT (-117.16840 32.71606),18579,6073005400,54.0,1541644.704,9342.847,61
19,132a465a-f85d-4df6-8456-c29ccfaaaabb,fc6cc685-eb0e-4afb-921f-e0da22b56c71,"[BICYCLE, PEDEVT]","IMAGE,VIDEO",CAMERA,32.71978112:-117.1690571,ONLINE,32.719781,-117.169057,POINT (-117.16906 32.71978),18579,6073005400,54.0,1541644.704,9342.847,61


Write tract geometries and joined assets to CARTO

In [14]:
to_carto(sd_tracts_gdf, 'san_diego_census_tracts', if_exists="replace")

Success! Data uploaded to table "san_diego_census_tracts" correctly


'san_diego_census_tracts'

In [15]:
to_carto(assets_within_tracts, 'san_diego_cityiq_assests', if_exists="replace")

Success! Data uploaded to table "san_diego_cityiq_assests" correctly


'san_diego_cityiq_assests'

## For the last month of data, aggregate hourly by assetid

*   Loop through each sensor and request events at that sensor for the last hour.  
*   Pull nested attributes out of "properties" and "measures" properties and store in their own attribute
*   Using tabulate, create a tabular representation of the events list 

In [17]:
# empty dataframe to collect hourly aggregate events
san_diego_pedestrian_hourly_aggregates = pd.DataFrame()


# loop through assets to fetch events for each asset
for index, row in san_diego_pedestrian_sensor_metadata_gdf.iterrows():
    for dt in rrule.rrule(rrule.HOURLY, dtstart=past, until=now):
        # empty list to collect events
        san_diego_pedestrian_sensor_events_list = []
        startTime = dt.timestamp() * 1000
        endTime_dt = dt + relativedelta(hours=1)
        endTime = endTime_dt.timestamp() * 1000
        myCIQ.fetchEvents("assets", row.assetUid, "PEDEVT", startTime, endTime, pageSize=100000)
        assetEvents = myCIQ.getEvents()
        # pull nested attributes out of "properties" and "measures" properties and store in their own attribute
        for a in assetEvents:
            a["latitude"] = row.latitude
            a["longitude"] = row.longitude
            a["directionUnit"] = a["properties"]["directionUnit"]
            a["speedUnit"] = a["properties"]["speedUnit"]
            a["eventUid"] = a["properties"]["eventUid"]
            a["counter_direction_speed"] = a["measures"]["counter_direction_speed"]
            a["counter_direction_pedestrianCount"] = a["measures"]["counter_direction_pedestrianCount"]
            a["pedestrianCount"] = a["measures"]["pedestrianCount"]
            a["counter_direction"] = a["measures"]["counter_direction"]
            a["speed"] = a["measures"]["speed"]
            a["direction"] = a["measures"]["direction"]
            san_diego_pedestrian_sensor_events_list.append(a)
        
        # with one hour of data make a dataframe, drop nulls, and group/aggregate pedestrian counts 
        san_diego_pedestrian_sensor_events_df = pd.DataFrame(san_diego_pedestrian_sensor_events_list)
        san_diego_pedestrian_sensor_events_df.dropna(inplace = True)
        
        # group by location ID to get a sum of pedestrianCounts 
        grouped_SD_ped_sensor_events_df = san_diego_pedestrian_sensor_events_df.groupby('assetUid').agg({'pedestrianCount': ['sum']})
        grouped_SD_ped_sensor_events_df.columns = ['pedestrianCount_sum']
        grouped_SD_ped_sensor_events_df.reset_index()
        grouped_SD_ped_sensor_events_df['startTime'] = startTime
        
        san_diego_pedestrian_hourly_aggregates.append(grouped_SD_ped_sensor_events_df, ignore_index=True)
        

san_diego_pedestrian_hourly_aggregates.head()

fetchEvents Failed
<Response [500]>
'content'


TypeError: 'NoneType' object is not iterable

### Write pedestrian count data to CARTO 

In [ ]:
to_carto(san_diego_pedestrian_hourly_aggregates, 'san_diego_pedestrian_hourly_aggregates', if_exists='replace')